In [1]:
!cp -r ../input/timm-pytorch /kaggle/working/
# !cp -r ../input/efficientnet-pytorch-07/* /kaggle/working/
!pip install /kaggle/working/timm-pytorch/timm-0.3.1-py3-none-any.whl
# !pip install /kaggle/working/efficientnet_pytorch-0.7.0

Processing ./timm-pytorch/timm-0.3.1-py3-none-any.whl


In [2]:
import numpy as np
import pandas as pd
import scipy.stats as stats

import albumentations as albu
from albumentations.pytorch import ToTensorV2

import cv2
import random
import os

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm

In [3]:
class config:
    use_amp = True
    use_tta = True
    ensemble = True
    n_fold = 5
    model_name = "tf_efficientnet_b4_ns"
    test_folder_path = "../input/cassava-leaf-disease-classification/test_images"
    train_folder_path = "../input/cassava-leaf-disease-classification/train_images"
    weight_path = "../input/weights-0126"
    weight_path2 = "../input/weights-0129"
    weight_path3 = "../input/weights-0203"
    weight_path4 = "../input/weights-0206"
    num_classes = 5
    device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
    batch_size = 10
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

In [4]:
class CustomModel(nn.Module):
    def __init__(self, model_name, pretrained=False, use_tta=False):
        super().__init__()
        self.use_tta = use_tta
        self.model = timm.create_model(model_name=model_name, pretrained=pretrained, num_classes=config.num_classes)

    def forward(self, x):
        if not self.use_tta:
            x = self.model(x)
        else:
            x0 = self.model(x[0])
            x1 = self.model(x[1])
            x2 = self.model(x[2])
#             x3 = self.model(x[3])
#             x4 = self.model(x[4])
            x = (x0 + x1 + x2) / 3.0
        return x

In [5]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


def get_transform(height, width):
    print(f"image_size: ({height}, {width})")
    transform = [
        albu.Resize(height=height, width=width),
        albu.Normalize(mean=config.mean, std=config.std),
        ToTensorV2(),
    ]
    return albu.Compose(transform)


def apply_only_one_augmentation(height, width, augmentation):
    transform = [
        augmentation,
        albu.Resize(height=height, width=width),
        albu.Normalize(mean=config.mean, std=config.std),
        ToTensorV2(),
    ]
    return albu.Compose(transform)


def apply_only_two_augmentation(height, width, augmentation1, augmentation2):
    transform = [
        albu.Resize(height=height, width=width),
        augmentation1,
        augmentation2,
        albu.Normalize(mean=config.mean, std=config.std),
        ToTensorV2(),
    ]
    return albu.Compose(transform)


def data_augmentation(image, transform):
    augmented = transform(image=image)
    return augmented["image"]


def get_ensemble_config():
    ensemble_dict = {
        "model_name": [config.model_name, config.model_name, config.model_name, config.model_name, config.model_name, config.model_name],
        "state_dict": [[torch.load(config.weight_path+f'/{config.model_name}_fold{fold}.pth') for fold in range(1, config.n_fold+1)], # 600x800
                      [torch.load(config.weight_path2+f'/{config.model_name}_fold{fold}_450x600.pth') for fold in range(1, config.n_fold+1)], # 450x600
                      [torch.load(config.weight_path2+f'/{config.model_name}_fold{fold}_750x1000.pth') for fold in range(1, config.n_fold+1)],# 750x1000
                      [torch.load(config.weight_path3+f'/{config.model_name}_fold{fold}_512x512.pth') for fold in range(1, config.n_fold+1)], # 512x512
                      [torch.load(config.weight_path3+f'/{config.model_name}_fold{fold}_900x1200.pth') for fold in range(1, config.n_fold+1)], # 900x1200
                      [torch.load(config.weight_path4+f'/{config.model_name}_fold{fold}_1024x1024.pth') for fold in range(1, config.n_fold+1)], # 1024x1024
                      ],
        }
    return ensemble_dict


def get_model(model_name, pretrained=False, use_tta=False):
    print(f"getting {model_name}..")
    model = CustomModel(model_name, pretrained=pretrained, use_tta=use_tta)
    model.to(config.device)
    return model


class CassavaDataset(Dataset):
    def __init__(self, df, path, transform, height, width, use_tta=False):
        self.df = df
        self.path = path
        self.transform = transform
        self.use_tta = use_tta
        self.height = height
        self.width = width

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        img_name = self.df.loc[idx, "image_id"]
        image = cv2.imread(self.path + "/" + img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.use_tta:
            image1 = data_augmentation(image, self.transform) # normal
            image2 = data_augmentation(image, apply_only_one_augmentation(self.height, self.width, albu.HorizontalFlip(p=1)))
            image3 = data_augmentation(image, apply_only_one_augmentation(self.height, self.width, albu.VerticalFlip(p=1)))
#             image4 = data_augmentation(image, apply_only_two_augmentation(self.height, self.width, albu.HorizontalFlip(p=1), albu.VerticalFlip(p=1)))
            return image1, image2, image3

        else:
            image = data_augmentation(image, self.transform)
            return image


def inference(model, states, test_loader, use_tta=False):
    probs = []
    with torch.no_grad():
        for images in test_loader:
            if use_tta:
                images = list(map(lambda x: x.to(config.device), images))
            else:
                images = images.to(config.device)
            avg_preds = []
            for state in states:
                model.load_state_dict(state)
                model.eval()
                
                if config.use_amp:
                    with torch.cuda.amp.autocast():
                        y_preds = model(images)
                else:
                    y_preds = model(images)
                avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
            avg_preds = np.mean(avg_preds, axis=0)
            probs.append(avg_preds)
        probs = np.concatenate(probs) # 配列を結合
    return probs


def judgement_preds(preds, confs):
    ret_preds = []
    for i in range(len(preds[0])):
        target_pred = list(preds[:,i])
        target_conf = list(confs[:,i])
        val = set(target_pred)
        if len(val) == 1: # 全部正解
            ret_preds.append(target_pred[0])
        elif len(val) == 2: # 3:3のケース
            ret_preds.append(pred_2diff(target_pred, target_conf))
        else:
            ret_preds.append(list(stats.mode(target_pred, axis=0)[0])[0])
    return ret_preds


def pred_2diff(pred, conf):
    max_value = max(conf)
    max_index = conf.index(max_value)
    return pred[max_index]

def pred_3diff(pred, conf):
    max_value = max(conf)
    max_index = conf.index(max_value)
    return pred[max_index]

In [6]:
DEBUG = False

seed = 42
seed_torch(seed=seed)


if DEBUG:
    sample_df = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
    display_num = 10
    sample_df = sample_df.head(display_num)
    print(sample_df.head(display_num))
    data_path = config.train_folder_path
else:
    sample_df = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
    data_path = config.test_folder_path

# fast sub
if sample_df.shape[0] != 1:
    if config.ensemble:
        ensemble_dict = get_ensemble_config()
        ensemble_predicts = None
        ensemble_confs = None
        image_size_dict = {
            "height": [600, 450, 750, 512, 900, 1024],
            "width": [800, 600, 1000, 512, 1200, 1024],
        }

        for idx, model_name in enumerate(ensemble_dict["model_name"]):
            model = get_model(model_name, pretrained=False, use_tta=config.use_tta)
            state_dicts = ensemble_dict["state_dict"][idx]
            test_dataset = CassavaDataset(sample_df, data_path, get_transform(height=image_size_dict["height"][idx], width=image_size_dict["width"][idx]), height=image_size_dict["height"][idx], width=image_size_dict["width"][idx], use_tta=config.use_tta)
            test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False, num_workers=4, pin_memory=True)
            _preds = inference(model, state_dicts, test_loader, use_tta=config.use_tta)
            preds = _preds.argmax(1)
            confs = _preds.max(1)
            if ensemble_predicts is None:
                ensemble_predicts = preds
                ensemble_confs = confs
            else:
                ensemble_predicts = np.vstack([ensemble_predicts, preds])
                ensemble_confs = np.vstack([ensemble_confs, confs])
#         ensemble_predicts = stats.mode(ensemble_predicts, axis=0)[0]
        ensemble_predicts = judgement_preds(ensemble_predicts, ensemble_confs)
        # submission
        sample_df['label'] = ensemble_predicts
    else:
        pass
        # model = get_model(config.model_name2, pretrained=False, use_tta=config.use_tta)
        # state_dicts = [torch.load(config.weight_path2+f'/{config.model_name2}_fold{fold}.pth') for fold in range(1, config.n_fold+1)]
        # test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)
        # predictions = inference(model, state_dicts, test_loader, use_tta=config.use_tta)
        # # submission
        # sample_df['label'] = predictions.argmax(1)
    
sample_df.to_csv("submission.csv", index=False)
sample_df.head(10)

,image_id,label
0,2216849948.jpg,4
